<a href="https://colab.research.google.com/github/camimariaf/Otimization_PS/blob/main/PS_HotelPart2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importações para o código poder criar e manipular objetos complexos, estabelecer contratos para classes e métodos, e manter uma tipagem clara e eficiente
from datetime import datetime
from typing import Dict, List, Optional
from enum import Enum
from dataclasses import dataclass
import uuid
from abc import ABC, abstractmethod

class Language(Enum):
    PORTUGUESE = "pt"
    ENGLISH = "en"
    SPANISH = "es"

class PaymentMethod(Enum):
    CREDIT_CARD = "credit_card"
    PIX = "pix"
    BANK_TRANSFER = "bank_transfer"

class PaymentStatus(Enum):
    PENDING = "pending"
    COMPLETED = "completed"
    FAILED = "failed"

# decorador para facilitar a criação de classes de dados
@dataclass
class Review:
    customer_id: str
    hotel_id: str
    rating: int
    comment: str
    date: datetime

@dataclass
class Room:
    room_number: str
    room_type: str
    base_price: float
    amenities: List[str]
    is_available: bool = True

class PaymentProcessor(ABC):
  # decorador para definir métodos abstratos em uma classe base abstrata (ABC)
    @abstractmethod
    def process_payment(self, amount: float) -> bool:
        pass

class CreditCardProcessor(PaymentProcessor):
    def process_payment(self, amount: float) -> bool:
        # simular processamento do cartão
        print(f"Processing credit card payment: ${amount:.2f}")
        return True

class PixProcessor(PaymentProcessor):
    def process_payment(self, amount: float) -> bool:
        # simular processamento do PIX
        print(f"Processing PIX payment: ${amount:.2f}")
        return True

class Customer:
    def __init__(self, name: str, email: str, phone: str, language: Language):
        self.id = str(uuid.uuid4())
        self.name = name
        self.email = email
        self.phone = phone
        self.language = language
        self.loyalty_points = 0
        self.bookings: List[str] = []
        self.preferences: Dict[str, str] = {}

    def add_loyalty_points(self, points: int):
        self.loyalty_points += points

    def update_preferences(self, preferences: Dict[str, str]):
        self.preferences.update(preferences)

class Hotel:
    def __init__(self, name: str, location: str, description: str):
        self.id = str(uuid.uuid4())
        self.name = name
        self.location = location
        self.description = description
        self.rooms: Dict[str, Room] = {}
        self.amenities: List[str] = []
        self.reviews: List[Review] = []
        self.photos: List[str] = []
        self.rating: float = 0.0

    def add_room(self, room: Room):
        self.rooms[room.room_number] = room

    def update_room_availability(self, room_number: str, is_available: bool):
        if room_number in self.rooms:
            self.rooms[room_number].is_available = is_available

    def add_review(self, review: Review):
        self.reviews.append(review)
        self._update_rating()

    def _update_rating(self):
        if self.reviews:
            self.rating = sum(review.rating for review in self.reviews) / len(self.reviews)

    def add_photo(self, photo_url: str):
        self.photos.append(photo_url)

class Booking:
    def __init__(self, customer_id: str, hotel_id: str, room_number: str,
                 check_in: datetime, check_out: datetime, total_price: float):
        self.id = str(uuid.uuid4())
        self.customer_id = customer_id
        self.hotel_id = hotel_id
        self.room_number = room_number
        self.check_in = check_in
        self.check_out = check_out
        self.total_price = total_price
        self.status = "confirmed"
        self.payment_status = PaymentStatus.PENDING

class BookingSystem:
    def __init__(self):
        self.customers: Dict[str, Customer] = {}
        self.hotels: Dict[str, Hotel] = {}
        self.bookings: Dict[str, Booking] = {}
        self.payment_processors: Dict[PaymentMethod, PaymentProcessor] = {
            PaymentMethod.CREDIT_CARD: CreditCardProcessor(),
            PaymentMethod.PIX: PixProcessor()
        }

    def register_customer(self, name: str, email: str, phone: str,
                         language: Language) -> Customer:
        customer = Customer(name, email, phone, language)
        self.customers[customer.id] = customer
        return customer

    def add_hotel(self, name: str, location: str, description: str) -> Hotel:
        hotel = Hotel(name, location, description)
        self.hotels[hotel.id] = hotel
        return hotel

    def create_booking(self, customer_id: str, hotel_id: str, room_number: str,
                      check_in: datetime, check_out: datetime) -> Optional[Booking]:
        if customer_id not in self.customers or hotel_id not in self.hotels:
            return None

        hotel = self.hotels[hotel_id]
        if room_number not in hotel.rooms or not hotel.rooms[room_number].is_available:
            return None

        # calcular preço total (simplificado)
        nights = (check_out - check_in).days
        total_price = hotel.rooms[room_number].base_price * nights

        booking = Booking(customer_id, hotel_id, room_number, check_in, check_out, total_price)
        self.bookings[booking.id] = booking
        hotel.update_room_availability(room_number, False)

        # adicionar pontos de fidelidade
        self.customers[customer_id].add_loyalty_points(int(total_price / 10))
        self.customers[customer_id].bookings.append(booking.id)

        return booking

    def cancel_booking(self, booking_id: str) -> bool:
        if booking_id not in self.bookings:
            return False

        booking = self.bookings[booking_id]
        hotel = self.hotels[booking.hotel_id]

        hotel.update_room_availability(booking.room_number, True)
        booking.status = "cancelled"
        return True

    def process_payment(self, booking_id: str, payment_method: PaymentMethod) -> bool:
        if booking_id not in self.bookings:
            return False

        booking = self.bookings[booking_id]
        processor = self.payment_processors[payment_method]

        if processor.process_payment(booking.total_price):
            booking.payment_status = PaymentStatus.COMPLETED
            return True

        booking.payment_status = PaymentStatus.FAILED
        return False

    def add_review(self, customer_id: str, hotel_id: str, rating: int, comment: str):
        if customer_id in self.customers and hotel_id in self.hotels:
            review = Review(customer_id, hotel_id, rating, comment, datetime.now())
            self.hotels[hotel_id].add_review(review)

    def generate_report(self) -> Dict:
        total_bookings = len(self.bookings)
        total_revenue = sum(booking.total_price for booking in self.bookings.values()
                          if booking.payment_status == PaymentStatus.COMPLETED)
        hotel_occupancy = {
            hotel.id: {
                'name': hotel.name,
                'occupancy_rate': sum(1 for room in hotel.rooms.values()
                                    if not room.is_available) / len(hotel.rooms) if hotel.rooms else 0,
                'average_rating': hotel.rating
            }
            for hotel in self.hotels.values()
        }

        return {
            'total_bookings': total_bookings,
            'total_revenue': total_revenue,
            'hotel_occupancy': hotel_occupancy
        }

def main():
    # inicializar sistema de reservas
    booking_system = BookingSystem()

    # exemplo para uso
    customer = booking_system.register_customer(
        "John Doe", "john@example.com", "123-456-7890", Language.ENGLISH
    )

    hotel = booking_system.add_hotel(
        "Grand Hotel", "New York", "Luxury hotel in downtown"
    )

    # adicioanr quartos ao hotel
    room = Room("101", "Deluxe", 200.0, ["WiFi", "TV", "Mini-bar"])
    hotel.add_room(room)

    # criar reserva
    check_in = datetime(2025, 3, 1)
    check_out = datetime(2025, 3, 5)
    booking = booking_system.create_booking(
        customer.id, hotel.id, "101", check_in, check_out
    )

    if booking:
        # provessar pagamente
        booking_system.process_payment(booking.id, PaymentMethod.CREDIT_CARD)

        # adicionar avaliação
        booking_system.add_review(customer.id, hotel.id, 5, "Excellent stay!")

        # gerar relatório
        report = booking_system.generate_report()
        print("\nBooking System Report:")
        print(f"Total Bookings: {report['total_bookings']}")
        print(f"Total Revenue: ${report['total_revenue']:.2f}")

if __name__ == "__main__":
    main()